In [1]:
# For EEG processing pipeline for one participant 
# Default and minimal pipeline, can be modified and extended
from mne import Epochs, find_events, Evoked
import mne
import numpy as np
from numpy import genfromtxt

%matplotlib qt

We load the data recorded by the Unicorn

In [9]:
my_data_s = genfromtxt('eeg_data.csv', delimiter=',', skip_header=1)

The data recorded is not in a usable format to be directly analyzed, we have to make some transformation to make a usable data object

In [10]:
# We keep only the usefull channel in our context (i.e., EEG channels and events)
my_data_s = my_data_s[:, [0, 1, 2, 3, 4, 5, 6, 7, 18]]
# We transpose the data from a column format into a line format
my_data_s = my_data_s.transpose()
# We can show the data here
my_data_s

In [11]:
sfreq = 250  # Sampling frequency of the unicorn 
ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'stim'] # Setup the types of each channel
ch_names = ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8', 'stim'] # Rename the channels with meaningful labels

# Create a raw MNE eeg data object in order to use the library for the analysis
montage = mne.channels.make_standard_montage('standard_1020')
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
raw = mne.io.RawArray(my_data_s, info)
raw.set_montage(montage)
# We have created the raw data object

In [12]:
# We can plot the sensors
raw.plot_sensors(show_names=True)

In [13]:
# Find the events
events = mne.find_events(raw, 'stim')


In [14]:
event_id = {'Stim 1': 1, 'Stim 2': 2, 'Stim 3': 3}

In [ ]:
# Plot raw data
raw.plot()
# Plot the power spectral density across channels
raw.plot_psd()

In [15]:
# Now we can apply a filter
raw.filter(1,40)
raw.notch_filter([60, 120])
# Plot the power spectral density after filtering
raw.plot_psd()

In [16]:
# and plot the continuous data 
raw.plot(n_channels=8, scalings=6e1, title='Filtered EEG data (1 - 40 Hz)',
         show=True)

In [17]:
# We can now create epochs based on the events (face, house)
# We setup the channels and take out the stimulation channel
picks = mne.pick_channels(ch_names, include=['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8'])
# We create an epochs object
epochs = Epochs(raw, events=events, event_id=event_id, picks=picks,
                tmin=-0.1, tmax=0.8, baseline=None, preload=True,
                verbose=False)

In [18]:
# We can now visualize the epochs and select bad epochs to be dropped
epochs.plot(scalings='auto')

In [19]:
# We can drop the selected epochs or channels
epochs.drop_bad()

In [20]:
# Show the new count
epochs

In [21]:
# We can now visualize the N170 for house
nave_stimeOne = len(epochs['Stim 1'])  # Number of averaged epochs
evoked_stimOne_data = np.mean(epochs['Stim 1'], axis=0) # Average the epochs 

# Create an Evoked data object
ch_types_evoked = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
ch_names_evoked = ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8']
info_evoked = mne.create_info(ch_names=ch_names_evoked, sfreq=sfreq, ch_types=ch_types_evoked)

evoked_stimeOne= mne.EvokedArray(evoked_stimOne_data, info=info_evoked, tmin=-0.1, nave=nave_stimeOne)
evoked_stimeOne.set_montage(montage)
# Apply baseline
evoked_stimeOne = evoked_stimeOne.apply_baseline((-.1, 0))

In [22]:
# Global field power 
evoked_stimeOne.plot(gfp=True, spatial_colors=True)

In [23]:
# Visualize
evoked_stimeOne.plot_joint(times = [.2, .5, .7])

In [24]:
# Topographies 
times = np.linspace(0.1, 0.8, 8)
evoked_stimeOne['Stim 1'].plot_topomap(ch_type='eeg', times=times, colorbar=True)

In [26]:
# mode = {‘pos’, ‘neg’, ‘abs’}
evoked_stimeOne.copy().crop(tmin=0.100, tmax=0.300).pick_channels(['Cz']).get_peak(ch_type='eeg', mode='neg',
                                             return_amplitude=True)
evoked_stimeOne.copy().crop(tmin=0.100, tmax=0.300).pick_channels(['Fz']).get_peak(ch_type='eeg', mode='neg',
                                             return_amplitude=True)

In [ ]:
# We can now visualize the N170 for face
nave_stimTwo = len(epochs['Stim 2'])  # Number of averaged epochs
evoked_stimTwo_data = np.mean(epochs['Stim 2'], axis=0) # Average the epochs 

# Create an Evoked data object
ch_types_evoked = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
ch_names_evoked = ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8']
info_evoked = mne.create_info(ch_names=ch_names_evoked, sfreq=sfreq, ch_types=ch_types_evoked)
evoked_stimTwo = mne.EvokedArray(evoked_stimTwo_data, info=info_evoked, tmin=-0.1, nave=nave_stimTwo)
evoked_stimTwo.set_montage(montage)
# Apply baseline
evoked_stimTwo = evoked_stimTwo.apply_baseline((-.1, 0))


In [ ]:
# Global field power 
evoked_stimTwo.plot(gfp=True, spatial_colors=True)

In [ ]:
# Visualize
evoked_stimTwo.plot_joint(times = [0.170])

In [ ]:
# Get max amplitude and latency
# mode = {‘pos’, ‘neg’, ‘abs’}
evoked_stimTwo.copy().crop(tmin=0.140, tmax=0.200).pick_channels(['Cz']).get_peak(ch_type='eeg', mode='neg',
                                             return_amplitude=True)



